# 场景二：多媒体资源的摘要

In [1]:
!ollama pull llama2-chinese:13b

pulling manifest
pulling 8359bebea988... 100% |██████████████████| (7.4/7.4 GB, 61 TB/s)        
pulling 65c6ec5c6ff0... 100% |████████████████████| (45/45 B, 955 kB/s)        
pulling dd36891f03a0... 100% |████████████████████| (31/31 B, 814 kB/s)        
pulling f94f529485e6... 100% |███████████████████| (382/382 B, 18 MB/s)        
verifying sha256 digest
writing manifest
removing any unused layers
success


In [ ]:
%pip install langchain langchain-core langchain-community
%pip install arxiv pymupdf

In [21]:
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载 arXiv 上的论文《ReAct: Synergizing Reasoning and Acting in Language Models》
loader = ArxivLoader(query="2210.03629", load_max_docs=1)
docs = loader.load()
print(docs[0].metadata)

# 把文本分割成 500 字一组的切片
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
chunks = text_splitter.split_documents(docs)

# 构建 Stuff 形态（即文本直接拼合）的总结链
doc_prompt = PromptTemplate.from_template("{page_content}")
chain = (
    {
        "content": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | PromptTemplate.from_template("用中文总结以下内容，不需要人物介绍，字数控制在 50 字以内：\n\n{content}")
    | ChatOllama(model="llama2-chinese:13b")
    | StrOutputParser()
)
# 由于论文很长，我们只选取前 2000 字作为输入并调用总结链
chain.invoke(chunks[:4])

{'Published': '2023-03-10', 'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models', 'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao', 'Summary': 'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.\naction plan generation) have primarily been studied as separate topics. In this\npaper, we explore the use of LLMs to generate both reasoning traces and\ntask-specific actions in an interleaved manner, allowing for greater synergy\nbetween the two: reasoning traces help the model induce, track, and update\naction plans as well as handle exceptions, while actions allow it to interface\nwith external sources, such as knowledge bases or environments, to gather\nadditional information. We apply our approach, named ReAct, to a diverse set of\nlanguage an

'\n这篇论文在ICLR 2023上发表，研究了如何兼顾理解和行动的能力。目前的大型语言模型(LLMs)已经成功地应用于许多语言理解和交互式决策任务，但它们的理解和行为两个方面主要是另外两个研究主题。我们将LLMs用于在逻辑追踪和任务特定的动作之间进行更加合作的使用，以此来产生更好的结果。我们命名为ReAct，并应用它到了多种语言和决策任务中，并取得了超越比较性好几个基线的表现。'

In [38]:
from functools import partial

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载 arXiv 上的论文《ReAct: Synergizing Reasoning and Acting in Language Models》
loader = ArxivLoader(query="2210.03629", load_max_docs=1)
docs = loader.load()

# 把文本分割成 500 字一组的切片
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
chunks = text_splitter.split_documents(docs)

llm = ChatOllama(model="llama2-chinese:13b")

# 构建工具函数：将 Document 转换成字符串
document_prompt = PromptTemplate.from_template("{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)

# 构建 Map 链：对每个文档都先进行一轮总结
map_chain = (
    {"context": partial_format_document}
    | PromptTemplate.from_template("Summarize this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

# 构建 Reduce 链：合并之前的所有总结内容
reduce_chain = (
    {"context": lambda strs: "\n\n".join(strs) }
    | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
    | llm
    | StrOutputParser()
)

# 把两个链合并成 MapReduce 链
map_reduce = map_chain.map() | reduce_chain
map_reduce.invoke(chunks[:4], config={"max_concurrency": 5})


'This paper introduces the REACT model which leverages both reasoning and acting abilities to improve the performance of large language models, achieving state-of-the-art results in various benchmarks. The authors present a novel approach that combines logic-based reasoning with behavior-based actions in LLMs, which enables them to better handle tasks such as question answering and text generation. Additionally, ReAct is an algorithm that combines chain-of-thought reasoning with simple Wikipedia API and generates human-like task-solving trajectories. It can outperform imitation and reinforcement learning methods in two interactive decision making benchmarks, achieving absolute success rates of 34% and 10%. Their proposed method is evaluated on several datasets and is shown to significantly outperform other baseline models, demonstrating its potential for improving the capabilities of language models. By using reasoning traces to help induce, track, and update action plans as well as ha

In [41]:
from functools import partial
from operator import itemgetter

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载 arXiv 上的论文《ReAct: Synergizing Reasoning and Acting in Language Models》
loader = ArxivLoader(query="2210.03629", load_max_docs=1)
docs = loader.load()

# 把文本分割成 500 字一组的切片
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
chunks = text_splitter.split_documents(docs)

llm = ChatOllama(model="llama2-chinese:13b")

# 构建工具函数：将 Document 转换成字符串
document_prompt = PromptTemplate.from_template("{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)

# 构建 Context 链：总结第一个文档并作为后续总结的上下文
first_prompt = PromptTemplate.from_template("Summarize this content:\n\n{context}")
context_chain = {"context": partial_format_document} | first_prompt | llm | StrOutputParser()

# 构建 Refine 链：基于上下文（上一次的总结）和当前内容进一步总结
refine_prompt = PromptTemplate.from_template(
    "Here's your first summary: {prev_response}. "
    "Now add to it based on the following context: {context}"
)
refine_chain = (
    {
        "prev_response": itemgetter("prev_response"),
        "context": lambda x: partial_format_document(x["doc"]),
    }
    | refine_prompt
    | llm
    | StrOutputParser()
)

# 构建一个负责执行 Refine 循环的函数 
def refine_loop(docs):
    summary = context_chain.invoke(docs[0])
    for i, doc in enumerate(docs[1:]):
        summary = refine_chain.invoke({"prev_response": summary, "doc": doc})
    return summary
  
refine_loop(chunks[:4])

"In this paper, we propose a novel approach called REACT that integrates reasoning traces and acting capabilities within a single framework to improve the overall performance of large language models (LLMs). By interleaving reasoning and acting, we can synergize the two cognitive abilities and enhance the capabilities of LLMs in various applications such as natural language processing, human-computer interaction, and cognitive systems.\n\nOur approach addresses issues of hallucination and error propagation in chain-of-thought reasoning by interacting with a simple Wikipedia API and generating human-like task-solving trajectories that are more interpretable than baselines without reasoning traces. Furthermore, on two interactive decision making benchmarks (ALFWorld and WebShop), ReAct outperforms imitation and reinforcement learning methods by an absolute success rate of 34% and 10% respectively, while being prompted with natural language commands.\n\nREACT's potential for improving the

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10, chunk_overlap=5)
print(text_splitter.split_text("你好LangChain实战"))
print(text_splitter.split_text("你好 LangChain 实战"))

['你好LangChai', 'gChain实战']
['你好', 'LangChain', '实战']


In [29]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

# 单个参数的函数可以直接被 RunnableLambda 封装
def length_function(text):
    return len(text)

# 多个个参数的函数需要先被封装成单个参数的函数，再传给 RunnableLambda
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOllama(model="llama2-chinese:13b")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='12')

In [31]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOllama(model="llama2-chinese:13b")

joke_chain = ChatPromptTemplate.from_template("讲一句关于{topic}的笑话") | model
poem_chain = ChatPromptTemplate.from_template("写一首关于{topic}的短诗") | model

# 通过 RunnableParallel（也可以叫做 RunnableMap）来并行执行两个调用链
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
map_chain.invoke({"topic": "小白兔"})

{'joke': AIMessage(content='有一只小白兔，他很喜欢吃花草。有天，他吃了太多的草丛了嘴子，家人看到他就开始笑起来说：“你好像被草踩住了！”小白兔半逼、半害羡地说：“是的，我已经吃完了。”'),
 'poem': AIMessage(content='嫩潤的小白兔\n在草地上玩耍。\n黑脸和白身，\n像小羊的朋友。\n')}